In [1]:
# Cleaning data into pandas dataframe format
import os
import pandas as pd

# read test data from the csv files in the result data folder
# put the data into dataframe and filter out desired variables 
dfRaw = pd.DataFrame()

for category in ['flowers', 'houses', 'paints']:
    directory = category + "/data/"

    for root,dirs,files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                dfParticipant = pd.read_csv(directory + file)
                
                # add category column
                dfParticipant['category'] = category             
                dfRaw = dfRaw.append(dfParticipant)

# filter out useful information from raw data
df = dfRaw[['key_resp_2.keys', 
            'key_resp_2.rt', 
            'image_left', 
            'image_right', 
            'trial_num', 
            'participant', 
            'date',
            'category']]


In [2]:
df

,key_resp_2.keys,key_resp_2.rt,image_left,image_right,trial_num,participant,date,category
0,z,5.310000,flowers/images/125.jpg,flowers/images/165.jpg,1,Ai,2020-11-21_22h07.13.545,flowers
1,m,4.539000,flowers/images/168.jpg,flowers/images/195.jpg,2,Ai,2020-11-21_22h07.13.545,flowers
2,z,1.846000,flowers/images/114.jpg,flowers/images/131.jpg,3,Ai,2020-11-21_22h07.13.545,flowers
3,z,1.767000,flowers/images/110.jpg,flowers/images/241.jpg,4,Ai,2020-11-21_22h07.13.545,flowers
4,z,2.042000,flowers/images/157.jpg,flowers/images/170.jpg,5,Ai,2020-11-21_22h07.13.545,flowers
...,...,...,...,...,...,...,...,...
406,z,1.165540,paints/images/236.jpg,paints/images/369.jpg,207,Zhaoliang Duan,2020-11-20_11h57.54.954,paints
407,z,2.876085,paints/images/219.jpg,paints/images/136.jpg,208,Zhaoliang Duan,2020-11-20_11h57.54.954,paints
408,z,0.760545,paints/images/306.jpg,paints/images/239.jpg,209,Zhaoliang Duan,2020-11-20_11h57.54.954,paints
409,m,0.985545,paints/images/108.jpg,paints/images/259.jpg,210,Zhaoliang Duan,2020-11-20_11h57.54.954,paints


In [3]:
#image = df_s['image_left'] + df_s['image_right']
image = list(df['image_left']) + list(df['image_right'])
len(image)

53284

In [5]:
# create score table for left image and right image, respectively
# the current score is calculated with basic average score of each
# image, more advanced math model can be applied to better differentiate 
# the performance of each image.
key = df['key_resp_2.keys']

leftScore = key.replace(['z','m'], [1,0])
rightScore = key.replace(['z','m'], [0,1])

# create image to score dataframe
imageDir = list(df['image_left']) + list(df['image_right'])
score = list(leftScore) + list(rightScore)
image_df = pd.DataFrame({'imageDir':imageDir, 'score':score})
image_df.groupby('imageDir').mean()

,score
imageDir,
flowers/images/0.jpg,0.363636
flowers/images/1.jpg,0.227273
flowers/images/10.jpg,0.431818
flowers/images/100.jpg,0.477273
flowers/images/101.jpg,0.454545
...,...
paints/images/95.jpg,0.363636
paints/images/96.jpg,0.500000
paints/images/97.jpg,0.568182


In [ ]:
# image resize for computation efficiency 
from PIL import Image
import os, sys

# demo folder 
path = "flowers/image_test/"
dirs = os.listdir( path )

for item in dirs:
    if os.path.isfile(path+item):
        im = Image.open(path+item)
        f, e = os.path.splitext(path+item)
        imResize = im.resize((200,200), Image.ANTIALIAS)
        imResize.save(f + 'resized.jpg', 'JPEG', quality=90)

In [95]:
# convert image to tricolor score array 
from PIL import Image
import numpy as np

def jpg_2_arr(file):
    img=Image.open(file)
    r,g,b=img.split()

    r_arr=np.array(r).reshape(650*500)
    g_arr=np.array(g).reshape(650*500)
    b_arr=np.array(b).reshape(650*500)

    img_arr=np.concatenate((r_arr,g_arr,b_arr))
    result=img_arr.reshape((3,650,500))
    return result

# demo image
jpg_2_arr('flowers/images/125.jpg')

975000

In [ ]:
# Next step:
# construct machine learning model (SVM classifier or KNN model) 
# reduce computation cost on sparse matrix
# improve model and train the existing data
# test model with big enough testing dataset
# predict the preference score for a given image.